<a href="https://colab.research.google.com/github/rmanzi13/Organizador_Catering/blob/main/Proveedores_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

In [ ]:
ruta_proveedores = ('proveedores_catering.json')
with open(ruta_proveedores,'r') as file:
  data = json.load(file)
df_proveedor = pd.DataFrame(data)

In [ ]:
df_proveedor.head()

,ID del Proveedor,Nombre del Proveedor,Tipo de Cocina,"Ubicación del proveedor(ciudad, dirección)",Contacto del proveedor,Lista especialidades,Precios (rango),Servicios de Bebidas (si o no),Capacidad del servicio (nº personas),Servicios extras
0,1,Madrid Catering,Internacional,"Calle Valdegovia 1, Madrid (Ciudad). Madrid",madridcatering@yahoo.com,"Cocina de Autor, Cocina Internacional",$150 - $200,Sí,Hasta 100 personas,No ofrece
1,2,Actividades De Restauranción Y Catering S.L,Internacional,"CALLE CALIDAD 34, BLOQUE 2, NAVE 4, Getafe. Ma...",actividadesderestauranciónycaterings.l@outlook...,"Cocina Internacional, Cocina Local",$150 - $200,Sí,Hasta 200 personas,"Alquiler de menaje y mobiliario, Música/DJ en ..."
2,3,Catering Lucía Y Joaquín,Internacional,"C/ Miguel Solas 9, Madrid (Ciudad). Madrid",cateringlucíayjoaquín@hotmail.com,"Cocina Vegetariana, Asados y Parrillas, Cocina...",$100 - $150,Sí,Hasta 200 personas,No ofrece
3,4,Gourmad Food,Internacional,"Modesto Lafuente 10, 7c, Madrid (Ciudad). Madrid",gourmadfood@outlook.com,"Tapas Variadas, Cocina Mediterránea",$100 - $150,Sí,Hasta 100 personas,No ofrece
4,5,Nuaset,Internacional,"Aravaca, Aravaca. Madrid",nuaset@outlook.com,"Cocina de Autor, Tapas Variadas",$100 - $150,Sí,Hasta 100 personas,"Show de cocina en vivo, Alquiler de menaje y m..."


In [ ]:
data_proveedor = df_proveedor[['Nombre del Proveedor','Lista especialidades','Precios (rango)']]

In [ ]:
data_proveedor = data_proveedor.copy()
data_proveedor.rename(columns={'Lista especialidades': 'Tipo_Catering', 'Precios (rango)':'Precios'}, inplace=True)

In [ ]:
data_proveedor.head()

,Nombre del Proveedor,Tipo_Catering,Precios
0,Madrid Catering,"Cocina de Autor, Cocina Internacional",$150 - $200
1,Actividades De Restauranción Y Catering S.L,"Cocina Internacional, Cocina Local",$150 - $200
2,Catering Lucía Y Joaquín,"Cocina Vegetariana, Asados y Parrillas, Cocina...",$100 - $150
3,Gourmad Food,"Tapas Variadas, Cocina Mediterránea",$100 - $150
4,Nuaset,"Cocina de Autor, Tapas Variadas",$100 - $150


In [ ]:
# Separamos las opciones en la columna 'Tipo_Catering'
data_proveedor['Tipo_Catering'] = data_proveedor['Tipo_Catering'].apply(lambda x: [tipo.strip() for tipo in x.split(',')])

In [ ]:
# Diversificamos las opciones en columnas individuales
df_catering = data_proveedor.explode('Tipo_Catering')

In [ ]:
# Reiniciar los índices del DataFrame
df_catering.reset_index(drop=True, inplace=True)

In [ ]:
df_catering.head()

,Nombre del Proveedor,Tipo_Catering,Precios
0,Madrid Catering,Cocina de Autor,$150 - $200
1,Madrid Catering,Cocina Internacional,$150 - $200
2,Actividades De Restauranción Y Catering S.L,Cocina Internacional,$150 - $200
3,Actividades De Restauranción Y Catering S.L,Cocina Local,$150 - $200
4,Catering Lucía Y Joaquín,Cocina Vegetariana,$100 - $150


In [ ]:
# Limpiar la columna 'Precios' y convertir a valores numéricos
df_catering['Precios'] = df_catering['Precios'].apply(lambda x: pd.to_numeric(x.replace('$', '').replace(',', '').strip('- ').split('-')[0]))

In [ ]:
df_catering.head(10)

,Nombre del Proveedor,Tipo_Catering,Precios
0,Madrid Catering,Cocina de Autor,150
1,Madrid Catering,Cocina Internacional,150
2,Actividades De Restauranción Y Catering S.L,Cocina Internacional,150
3,Actividades De Restauranción Y Catering S.L,Cocina Local,150
4,Catering Lucía Y Joaquín,Cocina Vegetariana,100
5,Catering Lucía Y Joaquín,Asados y Parrillas,100
6,Catering Lucía Y Joaquín,Cocina Asiática,100
7,Gourmad Food,Tapas Variadas,100
8,Gourmad Food,Cocina Mediterránea,100
9,Nuaset,Cocina de Autor,100


In [ ]:
df_catering.to_csv('df_catering_clena.csv', index = False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def catering_recommender(df_catering):
    try:
        # Limpiamos valores nulos o vacíos en las columnas utilizadas
        df_catering = df_catering.dropna(subset=['Tipo_Catering', 'Precios'])

        # Convertimos todas las columnas relevantes a cadenas antes de la concatenación
        df_catering['Tipo_Catering'] = df_catering['Tipo_Catering'].astype(str)
        df_catering['Precios'] = df_catering['Precios'].astype(str)


        # Concatenamos variables para formar la descripción del proveedor
        df_catering['proveedor_description'] = (
            df_catering['Tipo_Catering'].str.lower() + ' ' +
            df_catering['Precios'].str.lower()  # Convertir Capacidad_Num a cadena antes de concatenar
        )
        # Utilizamos CountVectorizer para convertir la descripción del proveedor en una matriz de términos
        vectorizer = CountVectorizer()
        proveedor_matrix = vectorizer.fit_transform(df_catering['proveedor_description'])

        # Calculamos la similitud del coseno entre las descripciones de proveedores
        similarity_matrix = cosine_similarity(proveedor_matrix, proveedor_matrix)

        return similarity_matrix

    except Exception as e:
        print(f"Error en catering_recommender: {e}")
        return pd.DataFrame()

# Ejemplo de uso
try:
    # Con los datos de data_catering
    matriz_similitud_proveedor = catering_recommender(df_catering)
    print(matriz_similitud_proveedor)
except Exception as e:
    print(f"Error en el ejemplo de uso: {e}")

[[1.         0.57735027 0.57735027 ... 0.57735027 0.28867513 0.57735027]
 [0.57735027 1.         1.         ... 0.66666667 0.33333333 0.66666667]
 [0.57735027 1.         1.         ... 0.66666667 0.33333333 0.66666667]
 ...
 [0.57735027 0.66666667 0.66666667 ... 1.         0.33333333 0.66666667]
 [0.28867513 0.33333333 0.33333333 ... 0.33333333 1.         0.33333333]
 [0.57735027 0.66666667 0.66666667 ... 0.66666667 0.33333333 1.        ]]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_recommendations_tokenized(matriz_similitud_cosine, df_catering, tipo_catering, precio_min, precio_max, umbral_similitud=0.5, top_n=5):
    try:
        # Filtramos el DataFrame según el tipo de catering y el rango de precios
        filtered_df = df_catering[(df_catering['Tipo_Catering'] == tipo_catering) &
                                  (df_catering['Precios'].astype(int) >= precio_min) &
                                  (df_catering['Precios'].astype(int) <= precio_max)]

         # Concatenamos variables para formar la descripción del proveedor
        df_catering['proveedor_description'] = (
           df_catering['Tipo_Catering'].str.lower() + ' ' +
           df_catering['Precios'].astype(str).str.lower()
        )

        # Obtenemos los índices de proveedores que cumplen con los criterios de filtro
        indices_proveedores = filtered_df.index.tolist()

        # Tokenizamos las descripciones utilizando CountVectorizer
        vectorizer = CountVectorizer()
        proveedor_matrix_tokenized = vectorizer.fit_transform(df_catering['proveedor_description'])

        # Calculamos la similitud del coseno entre los proveedores filtrados y todos los proveedores
        similarity_scores = cosine_similarity(proveedor_matrix_tokenized[indices_proveedores, :], proveedor_matrix_tokenized)

        # Calculamos la puntuación total de recomendación para cada proveedor
        scores = np.sum(similarity_scores, axis=0)

        # Aplicamos umbral de similitud
        relevant_indices = np.where(scores >= umbral_similitud)[0]

        # Ordenamos los proveedores según la puntuación de recomendación en orden descendente
        sorted_indices = np.argsort(scores[relevant_indices])[::-1]

        # Obtenemos los primeros top_n proveedores recomendados
        top_providers_indices = relevant_indices[sorted_indices[:top_n]]

        # Obtenemos nombres de proveedores recomendados
        recommended_providers = df_catering.iloc[top_providers_indices]['Nombre del Proveedor'].tolist()

        return recommended_providers

    except Exception as e:
        print(f"Error en get_recommendations_tokenized: {e}")
        return []

# Ejemplo de uso
try:
    tipo_catering_input = "Cocina Internacional"
    precio_min_input = 100
    precio_max_input = 200
    umbral_similitud_input = 0.6
    top_n_input = 5

    recomendaciones_tokenized = get_recommendations_tokenized(matriz_similitud_proveedor, df_catering, tipo_catering_input, precio_min_input, precio_max_input, umbral_similitud_input, top_n_input)

    if recomendaciones_tokenized:
        print(f"Proveedores recomendados para {tipo_catering_input} en el rango de precios {precio_min_input}-{precio_max_input} con umbral de similitud {umbral_similitud_input} y top {top_n_input} (tokenizado con CountVectorizer):")
        print(recomendaciones_tokenized)
    else:
        print("No se encontraron recomendaciones.")

except Exception as e:
    print(f"Error en el ejemplo de uso: {e}")


Proveedores recomendados para Cocina Internacional en el rango de precios 100-200 con umbral de similitud 0.6 y top 5 (tokenizado con CountVectorizer):
['Araceli Villalba', 'Señorío De Barguñó', 'Magnolia Catering', 'Ac Hotels', 'Modigliani Catering']
